In [1]:
import random
import time
import pygame

pygame.init()

# Настройки экрана и игры
FPS = 60
FramePerSec = pygame.time.Clock()
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5
SCORE = 0
COINS_COLLECTED = 0

# Цвета
BLUE  = (0, 0, 255)
RED   = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Шрифты
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, BLACK)

# Загрузка фона и изображений
background = pygame.image.load("AnimatedStreet.png")

# Экран
DISPLAYSURF = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
DISPLAYSURF.fill(WHITE)
pygame.display.set_caption("Racer")

# Классы
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__() 
        self.image = pygame.image.load("Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.bottom > SCREEN_HEIGHT:
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__() 
        self.image = pygame.image.load("Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)
    
    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0 and pressed_keys[pygame.K_LEFT]:
            self.rect.move_ip(-5, 0)
        if self.rect.right < SCREEN_WIDTH and pressed_keys[pygame.K_RIGHT]:
            self.rect.move_ip(5, 0)

class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("coin.png")
        self.image = pygame.transform.scale(self.image, (30, 30))
        self.rect = self.image.get_rect()
        self.reset_position()

    def reset_position(self):
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def move(self):
        self.rect.move_ip(0, SPEED)
        if self.rect.top > SCREEN_HEIGHT:
            self.reset_position()

# Создание объектов
P1 = Player()
E1 = Enemy()
coin = Coin()

# Группы спрайтов
enemies = pygame.sprite.Group(E1)
coins = pygame.sprite.Group(coin)
all_sprites = pygame.sprite.Group(P1, E1, coin)

# Таймер на увеличение скорости
INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 1000)

# Основной игровой цикл
def game_loop():
    global SPEED, SCORE, COINS_COLLECTED

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == INC_SPEED:
                SPEED += 0.5      
            if event.type == pygame.QUIT:
                running = False

        DISPLAYSURF.blit(background, (0, 0))

        # Отображение очков
        score_text = font_small.render("Score: " + str(SCORE), True, BLACK)
        coin_text = font_small.render("Coins: " + str(COINS_COLLECTED), True, BLACK)
        DISPLAYSURF.blit(score_text, (10, 10))
        DISPLAYSURF.blit(coin_text, (SCREEN_WIDTH - 100, 10))

        # Движение и отрисовка всех спрайтов
        for entity in all_sprites:
            entity.move()
            DISPLAYSURF.blit(entity.image, entity.rect)

        # Столкновение с врагом
        if pygame.sprite.spritecollideany(P1, enemies):
            pygame.mixer.Sound('crash.wav').play()
            time.sleep(1)
            DISPLAYSURF.fill(RED)
            DISPLAYSURF.blit(game_over, (30, 250))
            pygame.display.update()
            time.sleep(2)
            running = False

        # Сбор монеты
        if pygame.sprite.spritecollideany(P1, coins):
            COINS_COLLECTED += 1
            coin.reset_position()

        pygame.display.update()
        FramePerSec.tick(FPS)

    pygame.quit()

# Запуск игры
game_loop()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
